# Ver 1 using char-cnn model

In [3]:
from theano.sandbox import cuda
%matplotlib inline
from __future__ import division, print_function
import operator

import numpy as np
from keras.layers import TimeDistributed, Activation
from numpy.random import choice
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
from numpy.random import choice
import time
from keras.layers import Merge

In [4]:
# use entire text
text = open('./data/lgqm.txt').read()
text = unicode(text, "utf-8")
print('length of text: ', len(text))
print(text[:100])
charset = set(text)
char_freq_map = {c: 0 for c in charset}
for c in text:
    char_freq_map[c] += 1
char_freq_list = sorted(char_freq_map.items(), key=operator.itemgetter(1))
char_freq_list.reverse()
charlist = [pair[0] for pair in char_freq_list]
print('size of charset', len(charlist))
for i in range(40):
    print(charlist[i])
char2idx = dict((c, i) for i, c in enumerate(charlist))
text_in_idx = [char2idx[c] for c in text]
#text_in_idx[:24]
print(''.join(charlist[i] for i in text_in_idx[:24]))
maxlen = 48
sentences = []
next_chars = []
for i in range(0, len(text_in_idx) - maxlen):
    sentences.append(text_in_idx[i: i + maxlen])
    next_chars.append(text_in_idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))
sentences = np.array(sentences)
next_chars = np.array(next_chars)
sentences.shape, next_chars.shape
print('xxx done')

length of text:  9215340
《临高启明》吹牛者

严正声明：本书为丫丫小说网(www.shuyaya.com)的用户上传至其在本站的存储空间，本站只提供TXT全集电子书存储服务以及免费下载服务，以下作品内容之版权与本站无任何
size of charset 5688
 
，
的



。
是
了
一
不
“
”
这
人
有
在
他
来
个
大
上
就
得
到
要
们
和
说
里
―
也
着
子
下
还
出
过
地
会
道
《临高启明》吹牛者

严正声明：本书为丫丫小
nb sequences: 9215292
xxx done


In [5]:
# n_fac = 200
# batch_size = 64
# n_hidden = 512
# vocab_size = len(charlist)
# model = Sequential([
#         Embedding(vocab_size, n_fac, input_length=maxlen, batch_input_shape=(batch_size, maxlen)),
#         BatchNormalization(),
#         LSTM(n_hidden, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
#              consume_less='gpu'),
#         Dropout(0.2),
#         LSTM(n_hidden, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
#              consume_less='gpu'),
#         Dropout(0.2),
#         TimeDistributed(Dense(n_hidden, activation='relu')),
#         Dropout(0.2),
#         TimeDistributed(Dense(vocab_size, activation='softmax')),
# ])
# model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
# model.summary()
# model.optimizer.lr=0.01
# # model.load_weights('./data/text-generation-lgqm-ver-1-char-cnn-1497597351.55.h5')

In [11]:
n_fac = 200
batch_size = 64
vocab_size = len(charlist)
graph_in = Input ((vocab_size, n_fac))
convs = [ ] 
for fsz in range (3, 6): 
    x = Convolution1D(64, fsz, border_mode='same', activation="relu")(graph_in)
    x = MaxPooling1D()(x) 
    x = Flatten()(x) 
    convs.append(x)
out = Merge(mode="concat")(convs) 
graph = Model(graph_in, out)
model = Sequential ([
    Embedding(vocab_size, n_fac, input_length=maxlen, dropout=0.2, batch_input_shape=(batch_size, maxlen)),
    BatchNormalization(),
    graph,
    Dropout (0.5),
    Dense(n_hidden, activation='relu'),
    Dropout(0.2),
    TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()
model.optimizer.lr=0.01

AssertionError: 

In [8]:
def run_epochs(n):
    # input must be exact multiples of batch_size, for stateful
    # multiples = len(sentences)//batch_size*batch_size
    # sentences_truncated = sentences[:multiples]
    # next_chars_truncated = next_chars[:multiples]
    next_chars_adjusted = np.expand_dims(next_chars,-1)
    trn_size = batch_size * 100
    rand_trn_idx = np.random.randint(0, high=len(text)-1-trn_size)
    
    sentences_selected = sentences[rand_trn_idx:rand_trn_idx+trn_size]
    next_chars_selected = next_chars_adjusted[rand_trn_idx:rand_trn_idx+trn_size]
    
    for i in range(n):
        # model.reset_states()
        h = model.fit(sentences_selected, next_chars_selected, batch_size=batch_size, nb_epoch=1, shuffle=True)
        print(h.history, h.history['loss'])
        weight_file_name = './data/text-generation-lgqm-ver-1-char-cnn-'+str(time.time())+'.h5'
    model.save_weights(weight_file_name)
    print('saved weights: ', weight_file_name)

In [9]:
# NOTE: when cr is not together with lf, a line is biten! But, I doubt it ever happens in original text
# Better learn more to let it avoid naturally

def print_example_inline_seed_non_stateful():
    pre_string = u"第65535节 "
    
    rand_idx = np.random.randint(0, high=len(text)-maxlen-1)
    seed_string = text[rand_idx:rand_idx+maxlen]
    
    for i in range(maxlen+1):
        x=np.array([char2idx[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(charlist, p=preds)
        seed_string = seed_string + next_char
    seed_string = pre_string+seed_string[maxlen:]
    print(seed_string)
    for i in range(800):
        x=np.array([char2idx[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(charlist, p=preds)
        seed_string = seed_string + next_char
    print('final: ', seed_string)

In [10]:
run_epochs(1)

Exception: Error when checking model target: expected dense_4 to have 2 dimensions, but got array with shape (6400, 48, 1)

In [21]:
model.optimizer.lr=0.1
run_epochs(2)

Epoch 1/1
6400/6400 [==============================] - 41s - loss: 3.6252    
{'loss': [3.6252318596839905]} [3.6252318596839905]
saved weights:  ./data/text-generation-lgqm-ver-1-char-cnn-1497647977.22.h5
Epoch 1/1
6400/6400 [==============================] - 41s - loss: 1.7943    
{'loss': [1.7942678320407868]} [1.7942678320407868]
saved weights:  ./data/text-generation-lgqm-ver-1-char-cnn-1497648018.41.h5


In [24]:
model.optimizer.lr=0.001
run_epochs(1)

Epoch 1/1
6400/6400 [==============================] - 41s - loss: 3.3394    
{'loss': [3.3393771362304689]} [3.3393771362304689]
saved weights:  ./data/text-generation-lgqm-ver-1-char-cnn-1497648086.64.h5


In [25]:
model.optimizer.lr=0.01
run_epochs(1)

Epoch 1/1
6400/6400 [==============================] - 41s - loss: 3.5758    
{'loss': [3.5757711648941042]} [3.5757711648941042]
saved weights:  ./data/text-generation-lgqm-ver-1-char-cnn-1497648129.92.h5


In [26]:
run_epochs(1)

Epoch 1/1
6400/6400 [==============================] - 41s - loss: 3.4361    
{'loss': [3.436129505634308]} [3.436129505634308]
saved weights:  ./data/text-generation-lgqm-ver-1-char-cnn-1497648183.13.h5


In [28]:
print_example_inline_seed_non_stateful()

第65535节 她立刻叫“师傅开始工作大腿的城门口的地方转了一个主要市镇里，未会穿迁上唯后，所以老际已经全体或者部
final:  第65535节 她立刻叫“师傅开始工作大腿的城门口的地方转了一个主要市镇里，未会穿迁上唯后，所以老际已经全体或者部分书要人谈话已经堆到了办公室的几次关防在近的判决河、城内里却少了。

    左亚美了车站要道，我想耽搁没有交代。二个月是个庞代来。”随着列时也失去了自己曾经不再是经济下摆却，每天们面堆满了。

    “才员们……”左亚美意识到了。但是查询会，提出今天不是难公室，“晚上，虽然乘客们既然没证据不敢怠慢，面积控制小落――治安法庭的长度车。锦衣卫员出现了空心翼上的厢房。对于一个挂着来协理学院毕业”很少，交通便捷地。

    好在这段路在城铁，“成了进口，却说，一旦受刑架上审理案件，要么种得上一个人。”

    “她和你提出今天判决犯上参理员一面大量外迁。

    如何证明呢？她虽然也整修过警衔乎为一样中在城内的时候在一部分腿脚企划院的城门口还没有煤气灯照英份。

    夜间有着灯火的重要机构，她定很便在院子里还有个左亚美一些诸惊世骇俗，商馆。

    左亚美出了灯笼就更好，十鞭子两间闪亮很容易事情吗？百仞城内就在女首长那么短寒舌，领周边有个时候已经不够，她定在城里的南宝的事情。她好像东门市――捆在东门市的扩展也不能接发了。百仞城吞免。

    “我还没有独立人和做些辅助的地方正说。这里她很少来到了办公室。甚至紧挨，一路灯的李永薰微微鞠到近乎废墟。这衣服非常通亮，偌大的总务九小微弱的时候下，眼睛很得是什么在“和我是室友性命也没有警衔，就是按照英国的裙子堪称惊世骇俗的”李永薰微微鞠了县城里的还不需要这条临时的下力，刑务所看得控制的比较小。

    夜间们一起来就睡觉，我们一般的话已经拿到所以乘客了。她想耽密了。她把自己带上相起了车。虽然年龄比自己第一次领到县城里自己带左亚美出示证件命也没有公共马车通亮，现在是晚上专门的手段瞒不是经济一样的事情吗？”

    李永薰考虑再，毕竟地仆的裙子，后来看到这衣服，一裹圆住了涂乱院子里很少


In [ ]:
run_epochs(10)

Epoch 1/1
1152/6400 [====>.........................] - ETA: 34s - loss: 5.6278